In [1]:
import os
if 'jbook' in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../../..")))
from typing import Union

import numpy as np
import pandas as pd

from bcd.data_prep.base import DataPrep

In [2]:
%load -r 38-163 bcd/data_prep/case.py
# ------------------------------------------------------------------------------------------------ #
class CasePrep(DataPrep):
    """Performs Case metadata preparation."""

    def prep(
        self,
        calc_train_fp: str,
        calc_test_fp: str,
        mass_train_fp: str,
        mass_test_fp: str,
        case_fp: str,
        force: bool = False,
        result: bool = False,
    ) -> Union[None, pd.DataFrame]:
        """Combines training and test cases into a single csv case file.

        Args:
            calc_train_fp, calc_test_fp, mass_train_fp, mass_test_fp (str): The file paths to the
                calcification and mass training and test sets.
            case_fp (str): Path to output calcification and mass datasets.

            force (bool): Whether to force execution if output already exists. Default is False.
            result (bool): Whether the result should be returned. Default is False.

        Returns
            If result is True, the case dataframe is returned.
        """
        case_fp = os.path.abspath(case_fp)

        os.makedirs(os.path.dirname(case_fp), exist_ok=True)

        if force or not os.path.exists(case_fp):
            # Merge all case data into a single DataFrame
            df_cases = self._merge_cases(
                calc_train_fp=calc_train_fp,
                calc_test_fp=calc_test_fp,
                mass_train_fp=mass_train_fp,
                mass_test_fp=mass_test_fp,
            )

            df_cases = self._assign_mmg_id(df=df_cases)

            # Transform 'BENIGN WITHOUT CALLBACK' to 'BENIGN'
            df_cases["cancer"] = np.where(
                df_cases["pathology"] == "MALIGNANT", True, False
            )

            # Save datasets
            df_cases.to_csv(case_fp, index=False)

        if result:
            return df_cases

    def _merge_cases(
        self,
        calc_train_fp: str,
        calc_test_fp: str,
        mass_train_fp: str,
        mass_test_fp: str,
    ) -> pd.DataFrame:
        """Combines mass and calcification train and test files into a single file."""
        # Extracts absolute paths, a pre-emptive measure in case
        # jupyter book can't access the path
        calc_train_fp = os.path.abspath(calc_train_fp)
        calc_test_fp = os.path.abspath(calc_test_fp)
        mass_train_fp = os.path.abspath(mass_train_fp)
        mass_test_fp = os.path.abspath(mass_test_fp)

        df_calc_train = pd.read_csv(calc_train_fp)
        df_calc_test = pd.read_csv(calc_test_fp)
        df_mass_train = pd.read_csv(mass_train_fp)
        df_mass_test = pd.read_csv(mass_test_fp)

        # Add the filesets so that we can distinguish training
        # and test data
        df_calc_train["fileset"] = "training"
        df_calc_test["fileset"] = "test"
        df_mass_train["fileset"] = "training"
        df_mass_test["fileset"] = "test"

        # Replace spaces in column names with underscores.
        df_calc_train = self._format_column_names(df=df_calc_train)
        df_calc_test = self._format_column_names(df=df_calc_test)
        df_mass_train = self._format_column_names(df=df_mass_train)
        df_mass_test = self._format_column_names(df=df_mass_test)

        # Concatenate the files, ensuring that the morphologies are appropriately
        # set to NOT APPLICABLE where needed.
        df = pd.concat(
            [df_calc_train, df_calc_test, df_mass_train, df_mass_test], axis=0
        )
        df.loc[df["abnormality_type"] == "mass", "calc_type"] = "NOT APPLICABLE"
        df.loc[df["abnormality_type"] == "mass", "calc_distribution"] = "NOT APPLICABLE"
        df.loc[
            df["abnormality_type"] == "calcification", "mass_shape"
        ] = "NOT APPLICABLE"
        df.loc[
            df["abnormality_type"] == "calcification", "mass_margins"
        ] = "NOT APPLICABLE"
        return df

    def _assign_mmg_id(self, df: pd.DataFrame) -> pd.DataFrame:
        """Assign a mammogram id to each observation."""
        df["mmg_id"] = (
            df["abnormality_type"].apply(lambda x: x[0:4].capitalize())
            + "-"
            + df["fileset"].apply(lambda x: x.capitalize())
            + "_"
            + df["patient_id"]
            + "_"
            + df["left_or_right_breast"].upper()
            + "_"
            + df["image_view"]
        )

        return df

    def _format_column_names(self, df: pd.DataFrame) -> str:
        """Replaces spaces in column names with underscores."""

        def replace_columns(colname: str) -> str:
            return colname.replace(" ", "_")

        df.columns = df.columns.to_series().apply(replace_columns)
        return df